In [9]:
from op_analytics.datasources.chainsmeta.systemconfig.execute import execute_pull

df = execute_pull()

2025-06-12 00:42:07 [info     ] initialized duckdb at /var/folders/80/jqw6kmgj7j34j0x4mcs6hcl00000gn/T/iv7pcf74/op-analytics.duck.db filename=client.py lineno=118 process=6535
2025-06-12 00:42:07 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=6535
2025-06-12 00:42:07 [debug    ] loaded vault: 28 items         filename=vault.py lineno=79 process=6535
2025-06-12 00:42:07 [info     ] create duckddb gcs secret      filename=client.py lineno=66 process=6535
2025-06-12 00:42:07 [info     ] Reading data from 'chainsmeta/superchain_chain_list_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=94 process=6535
2025-06-12 00:42:08 [info     ] registered view: 'chainsmeta_superchain_chain_list_v1' using uri wildcard 'gs://oplabs-tools-data-sink/chainsmeta/superchain_chain_list_v1/dt=*/out.parquet' filename=client.py lineno=319 process=6535
┌─────────────────────────────────────┐
│                name                 │
│ 

In [2]:
from op_analytics.datasources.chainsmeta.systemconfig.endpoints import find_system_configs
from op_analytics.datasources.chainsmeta.systemconfig.systemconfig import SystemConfig
import requests

# # Get chains
# chains = find_system_configs()

# # Test OP Mainnet (first chain)
# test_chain = chains[0]  # OP Mainnet
# print(f"Testing: {test_chain.identifier}")
# print(f"System Config Proxy: {test_chain.system_config_proxy}")
# print(f"Chain RPC: {test_chain.rpc_url}")

In [3]:
# Create system config and get batch request
system_config = SystemConfig(system_config_proxy='0x229047fed2591dbec1eF1118d64F7aF3dB9EB290')
batch_request = system_config.rpc_batch()

print(f"\nBatch contains {len(batch_request)} requests")


Batch contains 34 requests


In [4]:
# Test with Ethereum RPC (since contracts are on L1)
ethereum_rpc = "https://ethereum-rpc.publicnode.com"
print(f"\nTesting with Ethereum RPC: {ethereum_rpc}")

response = requests.post(ethereum_rpc, json=batch_request, timeout=30)
response_data = response.json()

print(f"Response contains {len(response_data)} items")


Testing with Ethereum RPC: https://ethereum-rpc.publicnode.com
Response contains 34 items


In [5]:
response_data

[{'jsonrpc': '2.0',
  'id': 'block',
  'result': {'baseFeePerGas': '0x2fa589dc',
   'blobGasUsed': '0xa0000',
   'difficulty': '0x0',
   'excessBlobGas': '0x0',
   'extraData': '0x4275696c6465724e6574202842656176657229',
   'gasLimit': '0x2255077',
   'gasUsed': '0xa91677',
   'hash': '0x9d6fe70f7cb576841447d8bb0f5ca43e34a031aa1497c734a20c45b84764db1a',
   'logsBloom': '0x10e100076dd1ca0e22a488109f3047435b9d81010549201102a3061014118a50401885042b6a82920313c033080e0fb0a239c0228d03b43191a3224a4cbab10a8340b6b812415b8bf819c41ba07160a672e978d7a5491e24284680a79c44f037506070448a66c02aa25af1506124adc52c7a8640760606e01085121cd81b7041c4120f48b44bc62463cba16006ed58c414014c87d9f0004da0aa84485090260a3e0491d154b3fe8b1bdc5ce01f98bc187c45eb0553b589a0045302a4d906831502320c43137a741e132b3230603948c0372c0546c00b419f00a531724014208704f226e80ae844180447a99593c6c802294c68e401484cc04040a90249cc64aa',
   'miner': '0xdadb0d80178819f2319190d340ce9a924f783711',
   'mixHash': '0xb21d979ae352b67fba1e525510da8a9db95

In [6]:
def debug_detailed_decoding():
    """Show detailed decoding analysis of all system config fields"""
    # Get chains
    # chains = find_system_configs()
    
    # # Test OP Mainnet (first chain)
    # test_chain = chains[0]  # OP Mainnet
    # print(f"Testing: {test_chain.identifier}")
    # print(f"System Config Proxy: {test_chain.system_config_proxy}")
    
    # Create system config and get batch request
    system_config = SystemConfig(system_config_proxy="0x158Fd5715F16Ac1F2Dc959A299B383aAaf9B59EB")
    
    # Test with Ethereum RPC (since contracts are on L1)
    ethereum_rpc = "https://ethereum-rpc.publicnode.com"
    print(f"\nTesting with Ethereum RPC: {ethereum_rpc}")
    
    try:
        # Get the raw response first
        batch_request = system_config.rpc_batch()
        response = requests.post(ethereum_rpc, json=batch_request, timeout=30)
        response_data = response.json()
        
        # Get the decoded result
        result = system_config.call_rpc(rpc_endpoint=ethereum_rpc)
        
        if result is None:
            print("❌ call_rpc returned None")
            return
        
        # Create a mapping of raw data for comparison
        data_map = {}
        for item in response_data:
            if 'error' not in item and item['id'] != 'block':
                data_map[item['id']] = item['result']
        
        print(f"\n" + "="*80)
        print("DETAILED DECODING ANALYSIS - ALL FIELDS")
        print("="*80)
        
        # Show all the slot values (these stay as hex strings)
        print("\n📍 SLOT VALUES (remain as hex strings):")
        print("-" * 50)
        slot_fields = [
            ('batch_inbox_slot', 'BATCH_INBOX_SLOT'),
            ('dispute_game_factory_slot', 'DISPUTE_GAME_FACTORY_SLOT'), 
            ('l1_cross_domain_messenger_slot', 'L1_CROSS_DOMAIN_MESSENGER_SLOT'),
            ('l1_erc721_bridge_slot', 'L1_ERC_721_BRIDGE_SLOT'),
            ('l1_standard_bridge_slot', 'L1_STANDARD_BRIDGE_SLOT'),
            ('optimism_mintable_erc20_factory_slot', 'OPTIMISM_MINTABLE_ERC20_FACTORY_SLOT'),
            ('optimism_portal_slot', 'OPTIMISM_PORTAL_SLOT'),
            ('start_block_slot', 'START_BLOCK_SLOT'),
            ('unsafe_block_signer_slot', 'UNSAFE_BLOCK_SIGNER_SLOT'),
        ]
        
        for field_name, data_key in slot_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name}:")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            print()
        
        print("\n🔢 INTEGER VALUES (decoded from hex):")
        print("-" * 50)
        integer_fields = [
            ('version', 'version', 'uint256'),
            ('basefee_scalar', 'basefeeScalar', 'uint32'),
            ('blob_basefee_scalar', 'blobbasefeeScalar', 'uint64'),
            ('eip1559_denominator', 'eip1559Denominator', 'uint32'),
            ('eip1559_elasticity', 'eip1559Elasticity', 'uint32'),
            ('gas_limit', 'gasLimit', 'uint64'),
            ('maximum_gas_limit', 'maximumGasLimit', 'uint64'),
            ('minimum_gas_limit', 'minimumGasLimit', 'uint64'),
            ('operator_fee_constant', 'operatorFeeConstant', 'uint64'),
            ('operator_fee_scalar', 'operatorFeeScalar', 'uint32'),
            ('overhead', 'overhead', 'uint256'),
            ('scalar', 'scalar', 'uint256'),
        ]
        
        for field_name, data_key, uint_type in integer_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name} ({uint_type}):")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            if isinstance(decoded_value, int):
                print(f"    Hex:     0x{decoded_value:x}")
                print(f"    Bits:    {decoded_value.bit_length()} bits")
            print()
        
        print("\n📍 ADDRESS VALUES (decoded from hex):")
        print("-" * 50)
        address_fields = [
            ('batch_inbox', 'batchInbox'),
            ('dispute_game_factory', 'disputeGameFactory'),
            ('l1_cross_domain_messenger', 'l1CrossDomainMessenger'),
            ('l1_erc721_bridge', 'l1ERC721Bridge'),
            ('l1_standard_bridge', 'l1StandardBridge'),
            ('optimism_mintable_erc20_factory', 'optimismMintableERC20Factory'),
            ('optimism_portal', 'optimismPortal'),
            ('owner', 'owner'),
            ('start_block', 'startBlock'),
            ('unsafe_block_signer', 'unsafeBlockSigner'),
        ]
        
        for field_name, data_key in address_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name}:")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            print()
        
        print("\n🔗 HASH VALUES:")
        print("-" * 50)
        hash_fields = [
            ('batcher_hash', 'batcherHash'),
        ]
        
        for field_name, data_key in hash_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name}:")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            print()
        
        print("\n🏷️  SPECIAL FIELDS:")
        print("-" * 50)
        print(f"  version_hex:")
        print(f"    Value:   {result.version_hex}")
        print(f"    Type:    {type(result.version_hex)}")
        print(f"    (This is hex(version)[2:] = hex({result.version})[2:])")
        print()
        
        print(f"  contract_address:")
        print(f"    Value:   {result.contract_address}")
        print(f"    Type:    {type(result.contract_address)}")
        print()
        
        print(f"  block_number:")
        print(f"    Value:   {result.block_number}")
        print(f"    Type:    {type(result.block_number)}")
        print()
        
        print(f"  block_timestamp:")
        print(f"    Value:   {result.block_timestamp}")
        print(f"    Type:    {type(result.block_timestamp)}")
        print()
        
        print("\n⚠️  PROBLEMATIC LARGE VALUES FOR POLARS:")
        print("-" * 50)
        large_values = [
            ('version', result.version),
            ('overhead', result.overhead),
            ('scalar', result.scalar),
        ]
        
        max_uint64 = 2**64 - 1
        for name, value in large_values:
            print(f"  {name}: {value}")
            print(f"    Type: {type(value)}")
            print(f"    Bit length: {value.bit_length()} bits")
            print(f"    Max UInt64: {max_uint64}")
            print(f"    Fits in UInt64: {value <= max_uint64}")
            print(f"    Fits in UInt32: {value <= 2**32 - 1}")
            if value > max_uint64:
                print(f"    ❌ TOO LARGE FOR POLARS UInt64!")
            print()
        
        print("\n📋 COMPLETE DICTIONARY OUTPUT:")
        print("-" * 50)
        sample_dict = result.to_dict()
        for key, value in sample_dict.items():
            value_str = str(value)
            if len(value_str) > 60:
                value_str = value_str[:60] + "..."
            print(f"  '{key}': {value_str} (type: {type(value)})")
            
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

In [7]:
debug_detailed_decoding()


Testing with Ethereum RPC: https://ethereum-rpc.publicnode.com
2025-06-12 00:41:54 [debug    ] method operatorFeeConstant failed with -32000 for SystemConfig(system_config_proxy='0x158Fd5715F16Ac1F2Dc959A299B383aAaf9B59EB') filename=systemconfig.py lineno=199 process=6535
2025-06-12 00:41:54 [debug    ] method operatorFeeScalar failed with -32000 for SystemConfig(system_config_proxy='0x158Fd5715F16Ac1F2Dc959A299B383aAaf9B59EB') filename=systemconfig.py lineno=199 process=6535
2025-06-12 00:41:54 [info     ] methods ['operatorFeeConstant', 'operatorFeeScalar'] not supported by SystemConfig(system_config_proxy='0x158Fd5715F16Ac1F2Dc959A299B383aAaf9B59EB'), using default values filename=systemconfig.py lineno=219 process=6535

DETAILED DECODING ANALYSIS - ALL FIELDS

📍 SLOT VALUES (remain as hex strings):
--------------------------------------------------
  batch_inbox_slot:
    Raw:     0x71ac12829d66ee73d8d95bff50b3589745ce57edae70a3fb111a2342464dc597
    Decoded: 0x71ac12829d66ee73d8d